In [1]:


# Install groq library if not already installed
!pip install groq

import gradio as gr
import os
from groq import Groq

# Initialize Groq client - this will now be done inside the analyze_sentiment function
# The API key will be passed from the Gradio interface

def analyze_sentiment(api_key, text):
    """
    Analyzes the sentiment of the given text using Groq API
    Returns sentiment classification and confidence score
    """
    if not api_key.strip():
        return "Error: Groq API Key is required. Please enter your API key.", "", ""
    if not text.strip():
        return "Please enter some text to analyze.", "", ""

    try:
        # Initialize Groq client with the provided API key
        client = Groq(api_key=api_key)

        # Create the prompt for sentiment analysis
        prompt = f"""Analyze the sentiment of the following text and provide:
1. Overall sentiment (Positive, Negative, or Neutral)
2. Confidence score (0-100%)
3. Brief explanation of why you classified it this way

Text: "{text}"

Format your response exactly as:
SENTIMENT: [Positive/Negative/Neutral]
CONFIDENCE: [0-100]%
EXPLANATION: [Your explanation]"""

        # Call Groq API
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a sentiment analysis expert. Analyze text and provide clear, accurate sentiment classifications."
                },
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            model="llama-3.3-70b-versatile",
            temperature=0.3,
            max_tokens=500
        )

        # Parse the response
        response = chat_completion.choices[0].message.content

        # Extract sentiment, confidence, and explanation
        lines = response.strip().split('\n')
        sentiment = ""
        confidence = ""
        explanation = ""

        for line in lines:
            if line.startswith("SENTIMENT:"):
                sentiment = line.replace("SENTIMENT:", "").strip()
            elif line.startswith("CONFIDENCE:"):
                confidence = line.replace("CONFIDENCE:", "").strip()
            elif line.startswith("EXPLANATION:"):
                explanation = line.replace("EXPLANATION:", "").strip()

        # Color code the sentiment
        if "Positive" in sentiment:
            sentiment_display = f"✅ {sentiment}"
            color = "green"
        elif "Negative" in sentiment:
            sentiment_display = f"❌ {sentiment}"
            color = "red"
        else:
            sentiment_display = f"➖ {sentiment}"
            color = "gray"

        return sentiment_display, confidence, explanation

    except Exception as e:
        return f"Error: {str(e)}", "", "Please check your API key and try again."

# Create Gradio interface
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🤖 Sentiment Analysis Bot
        ### Powered by Groq API & Llama 3.3

        Enter any text to analyze its sentiment (positive, negative, or neutral).
        """
    )

    with gr.Row():
        with gr.Column():
            api_key_input = gr.Textbox(
                label="Enter your Groq API Key",
                type="password", # Use 'password' type to hide the key
                placeholder="Paste your Groq API key here...",
                lines=1
            )
            text_input = gr.Textbox(
                label="Enter text to analyze",
                placeholder="Type or paste your text here...",
                lines=5
            )
            analyze_btn = gr.Button("Analyze Sentiment", variant="primary")

            # Example texts
            gr.Examples(
                examples=[
                    ["I absolutely love this product! It exceeded all my expectations."],
                    ["This is the worst experience I've ever had. Completely disappointed."],
                    ["The weather today is cloudy with a chance of rain."],
                    ["I'm feeling pretty good about the meeting tomorrow."],
                    ["I hate waiting in long queues, it's so frustrating!"]
                ],
                inputs=text_input
            )

        with gr.Column():
            sentiment_output = gr.Textbox(label="Sentiment", interactive=False)
            confidence_output = gr.Textbox(label="Confidence Score", interactive=False)
            explanation_output = gr.Textbox(
                label="Explanation",
                lines=4,
                interactive=False
            )

    # Set up the button click event
    analyze_btn.click(
        fn=analyze_sentiment,
        inputs=[api_key_input, text_input],
        outputs=[sentiment_output, confidence_output, explanation_output]
    )

    gr.Markdown(
        """
        ---
        **Note:** Make sure to set your GROQ_API_KEY environment variable before running this app.
        Get your free API key at [console.groq.com](https://console.groq.com)
        """
    )

# Launch the app
if __name__ == "__main__":
    demo.launch()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.6 MB/s eta 0:00:00


/tmp/ipython-input-2175924025.py:89: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://29d8758790f2c0d345.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
